In [8]:
from smolagents import ToolCollection, ToolCallingAgent, OpenAIServerModel
from mcp import StdioServerParameters
# from pydantic import BaseModel, Field
import os
# from typing import Optional, List, Literal

### Configuration

In [9]:
os.environ['NEBIUS_API_KEY'] = open('secret.txt', 'r').read().strip()

In [10]:
server = StdioServerParameters(
    command="uvx",
    args=["opengenes-mcp", "stdio"]
)

In [11]:
MODEL = "Qwen/Qwen3-235B-A22B-Instruct-2507"

model = OpenAIServerModel(
    model_id=MODEL,
    api_key=os.environ["NEBIUS_API_KEY"],
    api_base="https://api.studio.nebius.com/v1/",
    temperature=0,
)

### Checks

In [16]:
with ToolCollection.from_mcp(
        server_parameters=server,
        trust_remote_code=True,
        structured_output=True
    ) as tools:
        # Проверить, что тулы действительно подхватились:
        print([t.name for t in tools.tools])

['opengenes_get_schema_info', 'opengenes_example_queries', 'opengenes_db_query']


In [115]:
def check_tool(tool):
    with ToolCollection.from_mcp(server, trust_remote_code=True, structured_output=False) as tc:
        # print([t.name for t in tc.tools])  # должны быть: opengenes_db_query, opengenes_get_schema_info, opengenes_example_queries
        ex = [t for t in tc.tools if t.name==tool][0]
        out = ex() 
        print(out)

In [116]:
check_tool("opengenes_example_queries")

[{"category":"Lifespan Effects - Ordered by Magnitude","description":"Genes that increase lifespan, ordered by greatest extension first","query":"SELECT HGNC, model_organism, effect_on_lifespan, lifespan_percent_change_mean FROM lifespan_change WHERE effect_on_lifespan = 'increases lifespan' AND lifespan_percent_change_mean IS NOT NULL ORDER BY lifespan_percent_change_mean DESC","key_concept":"Always order lifespan results by magnitude for relevance. Use LIMIT only when user specifically asks for 'top N' or similar"},{"category":"Lifespan Effects - Ordered by Magnitude","description":"Genes that decrease lifespan, ordered by greatest reduction first","query":"SELECT HGNC, model_organism, effect_on_lifespan, lifespan_percent_change_mean FROM lifespan_change WHERE effect_on_lifespan = 'decreases lifespan' AND lifespan_percent_change_mean IS NOT NULL ORDER BY lifespan_percent_change_mean ASC","key_concept":"Use ASC ordering for lifespan reductions to show largest decreases first. Use LIMI

In [117]:
check_tool("opengenes_get_schema_info")

{"database_overview":{"description":"OpenGenes database contains aging and lifespan research data with 4 main tables linked by HGNC gene symbols","total_tables":4,"primary_key":"HGNC (gene symbol) - links all tables together"},"critical_query_guidelines":{"multi_value_fields":{"description":"Some columns contain comma-separated values. ALWAYS use LIKE queries with wildcards for these fields.","fields":["gene_hallmarks.'hallmarks of aging' - contains multiple aging hallmarks per gene","lifespan_change.intervention_deteriorates - multiple biological processes that deteriorate","lifespan_change.intervention_improves - multiple biological processes that improve"],"example_syntax":"WHERE \"hallmarks of aging\" LIKE '%stem cell exhaustion%'"},"lifespan_metrics":{"description":"Database contains both mean and maximum lifespan change metrics. When user asks about lifespan effects without specifying, show both.","mean_vs_max":"lifespan_percent_change_mean shows average effect, lifespan_percent_

### Prompts

In [12]:
SYSTEM_PROMPT = open('opengenes_system_prompt.txt', 'r').read()

In [25]:
GENE = "NRF2"

USER_PROMPT = f"""
What are:
- Longevity/lifespan/healthspan association
- Modification effects
- Known genetic interventions
- Disease involvement
for the gene: {GENE} across different species?
If there are any known synonyms for the Gene use them to .
If some information is not available in the base for this name of the gene try to find synonyms and find and fetch data for them.
If this attampt alse fails return "Info not available".
"""

### Run

In [24]:
with ToolCollection.from_mcp(
    server_parameters=server,
    trust_remote_code=True,
    structured_output=False
) as tools:
    agent = ToolCallingAgent(
        model=model,
        tools=[*tools.tools],
        add_base_tools=False,
        max_steps=5,
    )
    agent.prompt_templates["system_prompt"] = SYSTEM_PROMPT
    result = agent.run(USER_PROMPT)

print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What are:                                                                                                       │
│ - Longevity/lifespan/healthspan association                                                                     │
│ - Modification effects                                                                                          │
│ - Known genetic interventions                                                                                   │
│ - Evolution                                                                                                     │
│ - Disease involvement                                                                                           │
│ for the gene: NRF2 across different species?                                                                    │
│ If there are any known synonyms for the Gene use them to .                                                      │
│ If some information is not available in the base for this name of the gene try to find synonyms and find and    │
│ fetch data for them.                                                                                            │
│ If this attampt alse fails return "Info not available".                                                         │
│                                                                                                                 │
╰─ OpenAIServerModel - Qwen/Qwen3-235B-A22B-Instruct-2507 ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'opengenes_get_schema_info' with arguments: {}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'opengenes_example_queries' with arguments: {}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: |{"category":"Lifespan Effects - Ordered by Magnitude","description":"Genes that increase lifespan, 
ordered by greatest extension first","query":"SELECT HGNC, model_organism, effect_on_lifespan, 
lifespan_percent_change_mean FROM lifespan_change WHERE effect_on_lifespan = 'increases lifespan' AND 
lifespan_percent_change_mean IS NOT NULL ORDER BY lifespan_percent_change_mean DESC","key_concept":"Always order 
lifespan results by magnitude for relevance. Use LIMIT only when user specifically asks for 'top N' or 
similar"},{"category":"Lifespan Effects - Ordered by Magnitude","description":"Genes that decrease lifespan, 
ordered by greatest reduction first","query":"SELECT HGNC, model_organism, effect_on_lifespan, 
lifespan_percent_change_mean FROM lifespan_change WHERE effect_on_lifespan = 'decreases lifespan' AND 
lifespan_percent_change_mean IS NOT NULL ORDER BY lifespan_percent_change_mean ASC","key_concept":"Use ASC ordering
for lifespan reductions to show largest decreases first. Use LIMIT only when user specifically asks for 'top N' or 
similar"},{"category":"Lifespan Effects - Mean vs Maximum","description":"Show both mean and maximum lifespan 
changes when user asks about lifespan effects","query":"SELECT HGNC, model_organism, effect_on_lifespan, 
lifespan_percent_change_mean, lifespan_percent_change_max, significance_mean, significance_max FROM lifespan_change
WHERE effect_on_lifespan = 'increases lifespan' AND (lifespan_percent_change_mean IS NOT NULL OR 
lifespan_percent_change_max IS NOT NULL) ORDER BY COALESCE(lifespan_percent_change_mean, 
lifespan_percent_change_max) DESC","key_concept":"IMPORTANT: When user asks about lifespan effects without 
specifying mean vs max, show both metrics. Researchers may be interested in either average effects or maximum 
potential. Use LIMIT only when user specifically asks for 'top N' or similar"},{"category":"Lifespan Effects - Mean
vs Maximum","description":"Compare mean vs maximum lifespan changes for the same interventions","query":"SELECT 
HGNC, model_organism, lifespan_percent_change_mean, lifespan_percent_change_max, (lifespan_percent_change_max - 
lifespan_percent_change_mean) as max_vs_mean_diff FROM lifespan_change WHERE lifespan_percent_change_mean IS NOT 
NULL AND lifespan_percent_change_max IS NOT NULL AND effect_on_lifespan = 'increases lifespan' ORDER BY 
max_vs_mean_diff DESC","key_concept":"Show the difference between maximum and mean effects to highlight variability
in responses. Use LIMIT only when user specifically asks for 'top N' or similar"},{"category":"Multi-Value Fields -
LIKE Queries","description":"Find genes associated with stem cell exhaustion hallmark","query":"SELECT HGNC, 
\"hallmarks of aging\" FROM gene_hallmarks WHERE \"hallmarks of aging\" LIKE '%stem cell 
exhaustion%'","key_concept":"CRITICAL: Always use LIKE with wildcards for multi-value 
fields"},{"category":"Multi-Value Fields - LIKE Queries","description":"Find interventions that improve 
cardiovascular system","query":"SELECT HGNC, intervention_improves, effect_on_lifespan, 
lifespan_percent_change_mean FROM lifespan_change WHERE intervention_improves LIKE '%cardiovascular system%' ORDER 
BY lifespan_percent_change_mean DESC","key_concept":"Use LIKE queries for intervention_improves and 
intervention_deteriorates columns"},{"category":"Multi-Value Fields - LIKE Queries","description":"Find genes 
affecting mitochondrial function","query":"SELECT HGNC, \"hallmarks of aging\" FROM gene_hallmarks WHERE 
\"hallmarks of aging\" LIKE '%mitochondrial%'","key_concept":"Multi-value hallmarks field requires LIKE pattern 
matching"},{"category":"Cross-Table Analysis","description":"Genes with both experimental lifespan effects and 
population longevity associations","query":"SELECT DISTINCT lc.HGNC, lc.effect_on_lifespan, lc.model_organism, 
la.ethnicity, la.\"study type\" FROM lifespan_change lc INNER JOIN longevity_associations la ON lc.HGNC = la.HGNC 
WHERE lc.effect_on_lifespan

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "The gene NRF2 (also known as NFE2L2) has been analyzed │
│ across multiple dimensions of aging and longevity research. Here is a comprehensive summary based on the        │
│ available data:\n\n### 1. Longevity/Lifespan/Healthspan Association\n- **Lifespan Effects**: No direct          │
│ experimental data from model organisms (e.g., mouse, C. elegans, fly) is available in the database for either   │
│ NRF2 or its synonym NFE2L2 regarding changes in lifespan (no entries in the `lifespan_change` table).\n-        │
│ **Healthspan Protection**: Despite the lack of lifespan extension data, NFE2L2 is strongly associated with      │
│ protection against age-related impairment. This suggests a role in improving healthspan rather than extending   │
│ maximum lifespan.\n\n### 2. Modification Effects\n- **Gene Activity Changes**: Alterations in NFE2L2 activity   │
│ are linked to protective effects against age-related functional decline. Specifically, changes in its activity  │
│ help protect against age-related impairments, indicating that upregulation or activation of this gene may be    │
│ beneficial for maintaining physiological function during aging.\n- **No detrimental effects** on lifespan or    │
│ healthspan were reported in the database.\n\n### 3. Known Genetic Interventions\n- There are **no recorded      │
│ genetic interventions** (e.g., knockout, overexpression) for NRF2/NFE2L2 in the database that have been tested  │
│ for effects on lifespan in model organisms.\n- Therefore, while NRF2 is a well-known target in aging research   │
│ (especially in oxidative stress response), no specific intervention outcomes are documented in this             │
│ dataset.\n\n### 4. Evolution\n- The database does not contain explicit evolutionary analysis or cross-species   │
│ comparison data for NRF2/NFE2L2.\n- However, given its role in fundamental cellular defense mechanisms (e.g.,   │
│ antioxidant response), NRF2 is highly conserved across species, which implies evolutionary importance in stress │
│ resistance and longevity pathways.\n\n### 5. Disease Involvement\n- NFE2L2 is involved in key aging-related     │
│ processes:\n  - **Oxidative stress regulation** (via 'accumulation of reactive oxygen species')\n  -            │
│ **Inflammation control** (linked to 'sterile inflammation')\n  - **Protein homeostasis** (via 'degradation of   │
│ proteolytic systems')\n  - **Transcriptional regulation** during aging ('transcriptional alterations')\n  -     │
│ **Cellular communication** ('intercellular communication impairment')\n- These associations suggest involvement │
│ in age-related diseases such as neurodegenerative disorders, cardiovascular diseases, and cancer, where         │
│ oxidative damage and chronic inflammation play critical roles.\n\n### Summary\nWhile **no direct lifespan       │
│ extension data** is available in the OpenGenes database for NRF2 (NFE2L2), it is clearly implicated in          │
│ **protecting against age-related functional decline** and is associated with multiple hallmarks of aging. Its   │
│ role appears to be more aligned with **healthspan improvement** than lifespan extension. The gene regulates     │
│ critical stress response pathways and is involved in mitigating oxidative damage and inflammation—key drivers   │
│ of aging and age-related diseases.\n\nThus, although experimental lifespan data is missing in this dataset, the │
│ biological evidence supports NRF2 as a significant player in aging biology through health-promoting             │
│ mechanisms."}                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: The gene NRF2 (also known as NFE2L2) has been analyzed across multiple dimensions of aging and 
longevity research. Here is a comprehensive summary based on the available data:

### 1. Longevity/Lifespan/Healthspan Association
- **Lifespan Effects**: No direct experimental data from model organisms (e.g., mouse, C. elegans, fly) is 
available in the database for either NRF2 or its synonym NFE2L2 regarding changes in lifespan (no entries in the 
`lifespan_change` table).
- **Healthspan Protection**: Despite the lack of lifespan extension data, NFE2L2 is strongly associated with 
protection against age-related impairment. This suggests a role in improving healthspan rather than extending 
maximum lifespan.

### 2. Modification Effects
- **Gene Activity Changes**: Alterations in NFE2L2 activity are linked to protective effects against age-related 
functional decline. Specifically, changes in its activity help protect against age-related impairments, indicating 
that upregulation or activation of this gene may be beneficial for maintaining physiological function during aging.
- **No detrimental effects** on lifespan or healthspan were reported in the database.

### 3. Known Genetic Interventions
- There are **no recorded genetic interventions** (e.g., knockout, overexpression) for NRF2/NFE2L2 in the database 
that have been tested for effects on lifespan in model organisms.
- Therefore, while NRF2 is a well-known target in aging research (especially in oxidative stress response), no 
specific intervention outcomes are documented in this dataset.

### 4. Evolution
- The database does not contain explicit evolutionary analysis or cross-species comparison data for NRF2/NFE2L2.
- However, given its role in fundamental cellular defense mechanisms (e.g., antioxidant response), NRF2 is highly 
conserved across species, which implies evolutionary importance in stress resistance and longevity pathways.

### 5. Disease Involvement
- NFE2L2 is involved in key aging-related processes:
  - **Oxidative stress regulation** (via 'accumulation of reactive oxygen species')
  - **Inflammation control** (linked to 'sterile inflammation')
  - **Protein homeostasis** (via 'degradation of proteolytic systems')
  - **Transcriptional regulation** during aging ('transcriptional alterations')
  - **Cellular communication** ('intercellular communication impairment')
- These associations suggest involvement in age-related diseases such as neurodegenerative disorders, 
cardiovascular diseases, and cancer, where oxidative damage and chronic inflammation play critical roles.

### Summary
While **no direct lifespan extension data** is available in the OpenGenes database for NRF2 (NFE2L2), it is clearly
implicated in **protecting against age-related functional decline** and is associated with multiple hallmarks of 
aging. Its role appears to be more aligned with **healthspan improvement** than lifespan extension. The gene 
regulates critical stress response pathways and is involved in mitigating oxidative damage and inflammation—key 
drivers of aging and age-related diseases.

Thus, although experimental lifespan data is missing in this dataset, the biological evidence supports NRF2 as a 
significant player in aging biology through health-promoting mechanisms.

Final answer: The gene NRF2 (also known as NFE2L2) has been analyzed across multiple dimensions of aging and 
longevity research. Here is a comprehensive summary based on the available data:

### 1. Longevity/Lifespan/Healthspan Association
- **Lifespan Effects**: No direct experimental data from model organisms (e.g., mouse, C. elegans, fly) is 
available in the database for either NRF2 or its synonym NFE2L2 regarding changes in lifespan (no entries in the 
`lifespan_change` table).
- **Healthspan Protection**: Despite the lack of lifespan extension data, NFE2L2 is strongly associated with 
protection against age-related impairment. This suggests a role in improving healthspan rather than extending 
maximum lifespan.

### 2. Modification Effects
- **Gene Activity Changes**: Alterations in NFE2L2 activity are linked to protective effects against age-related 
functional decline. Specifically, changes in its activity help protect against age-related impairments, indicating 
that upregulation or activation of this gene may be beneficial for maintaining physiological function during aging.
- **No detrimental effects** on lifespan or healthspan were reported in the database.

### 3. Known Genetic Interventions
- There are **no recorded genetic interventions** (e.g., knockout, overexpression) for NRF2/NFE2L2 in the database 
that have been tested for effects on lifespan in model organisms.
- Therefore, while NRF2 is a well-known target in aging research (especially in oxidative stress response), no 
specific intervention outcomes are documented in this dataset.

### 4. Evolution
- The database does not contain explicit evolutionary analysis or cross-species comparison data for NRF2/NFE2L2.
- However, given its role in fundamental cellular defense mechanisms (e.g., antioxidant response), NRF2 is highly 
conserved across species, which implies evolutionary importance in stress resistance and longevity pathways.

### 5. Disease Involvement
- NFE2L2 is involved in key aging-related processes:
  - **Oxidative stress regulation** (via 'accumulation of reactive oxygen species')
  - **Inflammation control** (linked to 'sterile inflammation')
  - **Protein homeostasis** (via 'degradation of proteolytic systems')
  - **Transcriptional regulation** during aging ('transcriptional alterations')
  - **Cellular communication** ('intercellular communication impairment')
- These associations suggest involvement in age-related diseases such as neurodegenerative disorders, 
cardiovascular diseases, and cancer, where oxidative damage and chronic inflammation play critical roles.

### Summary
While **no direct lifespan extension data** is available in the OpenGenes database for NRF2 (NFE2L2), it is clearly
implicated in **protecting against age-related functional decline** and is associated with multiple hallmarks of 
aging. Its role appears to be more aligned with **healthspan improvement** than lifespan extension. The gene 
regulates critical stress response pathways and is involved in mitigating oxidative damage and inflammation—key 
drivers of aging and age-related diseases.

Thus, although experimental lifespan data is missing in this dataset, the biological evidence supports NRF2 as a 
significant player in aging biology through health-promoting mechanisms.

[Step 4: Duration 15.61 seconds| Input tokens: 62,611 | Output tokens: 1,300]

The gene NRF2 (also known as NFE2L2) has been analyzed across multiple dimensions of aging and longevity research. Here is a comprehensive summary based on the available data:

### 1. Longevity/Lifespan/Healthspan Association
- **Lifespan Effects**: No direct experimental data from model organisms (e.g., mouse, C. elegans, fly) is available in the database for either NRF2 or its synonym NFE2L2 regarding changes in lifespan (no entries in the `lifespan_change` table).
- **Healthspan Protection**: Despite the lack of lifespan extension data, NFE2L2 is strongly associated with protection against age-related impairment. This suggests a role in improving healthspan rather than extending maximum lifespan.

### 2. Modification Effects
- **Gene Activity Changes**: Alterations in NFE2L2 activity are linked to protective effects against age-related functional decline. Specifically, changes in its activity help protect against age-related impairments, indicating that upregulation or activation of

### Test

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
from opengenes import run_query

In [ ]:
output = run_query("NRF2")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What are:                                                                                                       │
│             - Longevity/lifespan/healthspan association                                                         │
│             - Modification effects                                                                              │
│             - Known genetic interventions                                                                       │
│             - Disease involvement                                                                               │
│             for the gene: NRF2 across different species?                                                        │
│             If there are any known synonyms for the Gene use them to .                                          │
│             If some information is not available in the base for this name of the gene try to find synonyms and │
│ find and fetch data for them.                                                                                   │
│             If this attampt alse fails return "Info not available".                                             │
│                                                                                                                 │
╰─ OpenAIServerModel - Qwen/Qwen3-235B-A22B-Instruct-2507 ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'opengenes_get_schema_info' with arguments: {}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'opengenes_example_queries' with arguments: {}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: |{"category":"Lifespan Effects - Ordered by Magnitude","description":"Genes that increase lifespan, 
ordered by greatest extension first","query":"SELECT HGNC, model_organism, effect_on_lifespan, 
lifespan_percent_change_mean FROM lifespan_change WHERE effect_on_lifespan = 'increases lifespan' AND 
lifespan_percent_change_mean IS NOT NULL ORDER BY lifespan_percent_change_mean DESC","key_concept":"Always order 
lifespan results by magnitude for relevance. Use LIMIT only when user specifically asks for 'top N' or 
similar"},{"category":"Lifespan Effects - Ordered by Magnitude","description":"Genes that decrease lifespan, 
ordered by greatest reduction first","query":"SELECT HGNC, model_organism, effect_on_lifespan, 
lifespan_percent_change_mean FROM lifespan_change WHERE effect_on_lifespan = 'decreases lifespan' AND 
lifespan_percent_change_mean IS NOT NULL ORDER BY lifespan_percent_change_mean ASC","key_concept":"Use ASC ordering
for lifespan reductions to show largest decreases first. Use LIMIT only when user specifically asks for 'top N' or 
similar"},{"category":"Lifespan Effects - Mean vs Maximum","description":"Show both mean and maximum lifespan 
changes when user asks about lifespan effects","query":"SELECT HGNC, model_organism, effect_on_lifespan, 
lifespan_percent_change_mean, lifespan_percent_change_max, significance_mean, significance_max FROM lifespan_change
WHERE effect_on_lifespan = 'increases lifespan' AND (lifespan_percent_change_mean IS NOT NULL OR 
lifespan_percent_change_max IS NOT NULL) ORDER BY COALESCE(lifespan_percent_change_mean, 
lifespan_percent_change_max) DESC","key_concept":"IMPORTANT: When user asks about lifespan effects without 
specifying mean vs max, show both metrics. Researchers may be interested in either average effects or maximum 
potential. Use LIMIT only when user specifically asks for 'top N' or similar"},{"category":"Lifespan Effects - Mean
vs Maximum","description":"Compare mean vs maximum lifespan changes for the same interventions","query":"SELECT 
HGNC, model_organism, lifespan_percent_change_mean, lifespan_percent_change_max, (lifespan_percent_change_max - 
lifespan_percent_change_mean) as max_vs_mean_diff FROM lifespan_change WHERE lifespan_percent_change_mean IS NOT 
NULL AND lifespan_percent_change_max IS NOT NULL AND effect_on_lifespan = 'increases lifespan' ORDER BY 
max_vs_mean_diff DESC","key_concept":"Show the difference between maximum and mean effects to highlight variability
in responses. Use LIMIT only when user specifically asks for 'top N' or similar"},{"category":"Multi-Value Fields -
LIKE Queries","description":"Find genes associated with stem cell exhaustion hallmark","query":"SELECT HGNC, 
\"hallmarks of aging\" FROM gene_hallmarks WHERE \"hallmarks of aging\" LIKE '%stem cell 
exhaustion%'","key_concept":"CRITICAL: Always use LIKE with wildcards for multi-value 
fields"},{"category":"Multi-Value Fields - LIKE Queries","description":"Find interventions that improve 
cardiovascular system","query":"SELECT HGNC, intervention_improves, effect_on_lifespan, 
lifespan_percent_change_mean FROM lifespan_change WHERE intervention_improves LIKE '%cardiovascular system%' ORDER 
BY lifespan_percent_change_mean DESC","key_concept":"Use LIKE queries for intervention_improves and 
intervention_deteriorates columns"},{"category":"Multi-Value Fields - LIKE Queries","description":"Find genes 
affecting mitochondrial function","query":"SELECT HGNC, \"hallmarks of aging\" FROM gene_hallmarks WHERE 
\"hallmarks of aging\" LIKE '%mitochondrial%'","key_concept":"Multi-value hallmarks field requires LIKE pattern 
matching"},{"category":"Cross-Table Analysis","description":"Genes with both experimental lifespan effects and 
population longevity associations","query":"SELECT DISTINCT lc.HGNC, lc.effect_on_lifespan, lc.model_organism, 
la.ethnicity, la.\"study type\" FROM lifespan_change lc INNER JOIN longevity_associations la ON lc.HGNC = la.HGNC 
WHERE lc.effect_on_lifespan

Observations: {"database_overview":{"description":"OpenGenes database contains aging and lifespan research data 
with 4 main tables linked by HGNC gene symbols","total_tables":4,"primary_key":"HGNC (gene symbol) - links all 
tables together"},"critical_query_guidelines":{"multi_value_fields":{"description":"Some columns contain 
comma-separated values. ALWAYS use LIKE queries with wildcards for these 
fields.","fields":|"gene_hallmarks.'hallmarks of aging' - contains multiple aging hallmarks per 
gene","lifespan_change.intervention_deteriorates - multiple biological processes that 
deteriorate","lifespan_change.intervention_improves - multiple biological processes that 
improve"],"example_syntax":"WHERE \"hallmarks of aging\" LIKE '%stem cell 
exhaustion%'"},"lifespan_metrics":{"description":"Database contains both mean and maximum lifespan change metrics. 
When user asks about lifespan effects without specifying, show both.","mean_vs_max":"lifespan_percent_change_mean 
shows average effect, lifespan_percent_change_max shows maximum observed effect","when_to_show_both":"If user asks 
about 'lifespan effects' or 'lifespan changes' without specifying mean vs max, include both 
metrics","ordering_both":"Use COALESCE(lifespan_percent_change_mean, lifespan_percent_change_max) for ordering when
showing both","significance":"Both mean and max have corresponding significance columns (significance_mean, 
significance_max)"},"result_ordering":{"lifespan_extension":"ORDER BY lifespan_percent_change_mean DESC (highest 
increase first)","lifespan_reduction":"ORDER BY lifespan_percent_change_mean ASC (largest decrease 
first)","importance":"Always order lifespan results by magnitude of effect for relevance","both_metrics":"When 
showing both mean and max, use COALESCE for ordering or show 
comparison"},"comprehensive_aging_evidence":{"description":"For questions about aging evidence, link to aging, or 
longevity associations for a gene, query ALL 4 tables for complete evidence","required_tables":"1) gene_criteria 
(aging-related criteria), 2) gene_hallmarks (aging pathways), 3) lifespan_change (experimental effects), 4) 
longevity_associations (human population studies)","example_patterns":"Evidence of X and aging, Link between X and 
aging, X gene aging associations, What evidence links X to aging","critical_note":"Do NOT omit 
longevity_associations table - it contains crucial human population genetics data"},"gene_queries":"Use HGNC column
for gene symbols (TP53, FOXO3, etc.)","safety":"Only SELECT queries allowed - no INSERT, UPDATE, DELETE, or DDL 
operations"},"tables":{"lifespan_change":{"description":{"purpose":"Experimental data on how gene modifications 
affect lifespan in various model organisms","key_columns":"HGNC, model_organism, effect_on_lifespan, intervention 
methods, lifespan measurements","use_cases":"Questions about gene effects on lifespan, experimental conditions, 
organism studies","special_notes":"Contains multi-value fields for intervention effects. Use LIKE queries for 
intervention_deteriorates and intervention_improves 
columns."},"columns":|{"name":"HGNC","type":"TEXT","nullable":true,"primary_key":false,"description":"Gene symbol 
(standard gene names like TP53, 
FOXO3)"},{"name":"model_organism","type":"TEXT","nullable":true,"primary_key":false,"description":"Organism used 
for experiment (mouse, C. elegans, fly, 
etc.)"},{"name":"sex","type":"TEXT","nullable":true,"primary_key":false,"description":"Sex of organism used (male, 
female, all, hermaphrodites, 
etc.)"},{"name":"line","type":"TEXT","nullable":true,"primary_key":false,"description":""},{"name":"effect_on_lifes
pan","type":"TEXT","nullable":true,"primary_key":false,"description":"Direction of lifespan change 
(increases/decreases/no 
change)"},{"name":"control_cohort_size","type":"REAL","nullable":true,"primary_key":false,"description":""},{"name"
:"experiment_cohort_size","type":"REAL","nullable":true,"primary_key":false,"description":""},{"name":"

[Step 1: Duration 1.93 seconds| Input tokens: 9,474 | Output tokens: 34]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
output